In [ ]:
from tune_util import *
import numpy as np
import pickle
import functools
import cv2

import tempfile


In [ ]:
import random
import configparser
import os
episodes_config = configparser.ConfigParser(
    allow_no_value=True, inline_comment_prefixes=";"
)
episodes_config.read(os.path.join(os.getcwd(), "params/episode_params_val.ini"))
series = episodes_config.sections()[1:]

In [ ]:
def_num = np.array([10,1,3,1000,4,1,0.8,2,0.35,2,3])
def_num_log = np.array(list(np.log(def_num)) + [0.0])

In [ ]:
my_log = []
res = exec_seqs(def_num_log,'test_eval',['t_ETH1','--render'],log_file=my_log)
res

In [ ]:
import cma

In [ ]:
cv2.destroyAllWindows()

REND_FPS = 10
rend_time_delay = int(round(1000/REND_FPS))

es = cma.CMAEvolutionStrategy(def_num_log,0.1)
opt_iter = 0
best_subsets = []
all_res = []

temp_dir = tempfile.TemporaryDirectory()

while not es.stop():
    cv2.destroyAllWindows()
    opt_iter += 1

    solutions = es.ask()
    #r_vals = []
    chosen = [random.choice(series),'--render']
    #chosen = ['t_ETH1','--render']

    for sol_i,params in enumerate(solutions):
        res = exec_seqs(params,'mancma_{:03d}'.format(sol_i),chosen,log_file=my_log)
        #r_vals.append(res)
    
    ind_r = []
    for sol_i,i in enumerate(sorted(glob.glob(os.path.join(os.getcwd(),'tests/socnav/mancma_*')))):
        # scary what if multiple run!?!
        params = solutions[sol_i]
        
        pkl = glob.glob(os.path.join(i,'*/*.pkl'))[0]
        with open(pkl,'rb') as fp:
            res = pickle.load(fp)

        cost, costs_vec = eval_heuristic(res)

        jpgs = sorted(glob.glob(pkl.replace('episode_score_','').replace('.pkl','_obs_obs*.jpg')))
        read_data = [cv2.imread(_) for _ in jpgs]
        ind_r.append((cost,params,res,read_data))
    pair_comps = []
    def compare(i,j):
        cv2.destroyAllWindows()

        c1,_,_,set1 = ind_r[i]
        c2,_,_,set2 = ind_r[j]

        s12_s = list(set1[0].shape)
        s12_s[1]*=2
        while True:
            cv2.imshow("Comp {:.1f} {:.1f}".format(c1,c2), 
                       np.stack([set1[len(set1)-1],set2[len(set2)-1]],axis=1).reshape(s12_s))
            a = cv2.waitKey(rend_time_delay)

            if a == ord('1'):
                pair_comps.append([1,i,j])
                return 1
            if a == ord('2'):
                pair_comps.append([2,j,i])
                return -1
            if a == ord('3'):
                pair_comps.append([3] + ([i,j] if (c1 < c2) else [j,j]))
                return 2*(c1 <c2)-1
            if a == ord('q'):
                return
            if a == ord('g'):
                break
                
        while True:
            for k in range(max(len(set1),len(set2))):
                cv2.imshow("Comp {:.1f} {:.1f}".format(c1,c2), 
                           np.stack([set1[min(k,len(set1)-1)],set2[min(k,len(set2)-1)]],axis=1).reshape(s12_s))
                a = cv2.waitKey(rend_time_delay)

                if a == ord('1'):
                    pair_comps.append([1,i,j])
                    return 1
                if a == ord('2'):
                    pair_comps.append([2,j,i])
                    return -1
                if a == ord('3'):
                    pair_comps.append([3] + ([i,j] if (c1 < c2) else [j,j]))
                    return 2*(c1 <c2)-1
                if a == ord('q'):
                    return
                else:
                    continue
    res = sorted(list(range(len(ind_r))),key=functools.cmp_to_key(compare))
    
    batch_res = { 'cost':[_[0] for _ in ind_r],
      'params': [_[1] for _ in ind_r],
      'pkl': [_[2] for _ in ind_r],
      'order': res,
      'comps': pair_comps,
    }
    all_res.append(batch_res)
    
    with open(os.path.join(temp_dir.name,'{:03d}.pkl'.format(opt_iter)),'wb') as fp:              
        pickle.dump(ind_r[res[-1]], fp)

    t_vals = np.arange(len(solutions))/len(solutions)-opt_iter
    t_vals = t_vals[::-1] # rewards become costs
    r_vals = []
    for i in sorted(range(len(solutions))):
        v = t_vals[res.index(i)]
        r_vals.append(v)
    es.tell(solutions, r_vals)
    es.logger.add()
    es.disp()

In [ ]:
import glob
ind_best = []
for bf in sorted(glob.glob(os.path.join(temp_dir.name,'*.pkl'))):
    with open(bf,'rb') as fp:              
        ind_best.append(pickle.load(fp))

In [ ]:
pair_comps = []
def compare_best(i,j):
    cv2.destroyAllWindows()

    c1,_,_,set1 = ind_best[i]
    c2,_,_,set2 = ind_best[j]

    s12_s = list(set1[0].shape)
    s12_s[1]*=2
    
    while True:
        cv2.imshow("Comp {:.1f} {:.1f}".format(c1,c2), 
                   np.stack([set1[len(set1)-1],set2[len(set2)-1]],axis=1).reshape(s12_s))
        a = cv2.waitKey(rend_time_delay)

        if a == ord('1'):
            pair_comps.append([1,i,j])
            return 1
        if a == ord('2'):
            pair_comps.append([2,j,i])
            return -1
        if a == ord('3'):
            pair_comps.append([3] + ([i,j] if (c1 < c2) else [j,j]))
            return 2*(c1 <c2)-1
        if a == ord('q'):
            return
        if a == ord('g'):
            break
    while True:
        for k in range(max(len(set1),len(set2))):
            cv2.imshow("Comp {:.1f} {:.1f}".format(c1,c2), 
                       np.stack([set1[min(i,len(set1)-1)],set2[min(i,len(set2)-1)]],axis=1).reshape(s12_s))
            a = cv2.waitKey(30)

            if a == ord('1'):
                pair_comps.append([1,i,j])
                return 1
            if a == ord('2'):
                pair_comps.append([2,j,i])
                return -1
            if a == ord('3'):
                pair_comps.append([3] + ([i,j] if (c1 < c2) else [j,j]))
                return 2*(c1 <c2)-1
            if a == ord('q'):
                return
            else:
                continue
res = sorted(list(range(len(ind_r))),key=functools.cmp_to_key(compare_best))

batch_res = { 'cost':[_[0] for _ in ind_best],
  'params': [_[1] for _ in ind_best],
  'pkl': [_[2] for _ in ind_best],
  'order': res,
  'comps': pair_comps,
}
all_res.append(batch_res)


In [ ]:
results = {'cfg':ind_best[res[-1]][1],'log':my_log,'user_batches':all_res,'final_res':res}
import glob
import pickle
N=len(glob.glob('mancma_results*.pkl'))
with open('mancma_results_{}.pkl'.format(N),'wb') as fp:
    pickle.dump(results,fp)